In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = r"C:\AGROTECH\processed_images"
validation_dir = r"C:\AGROTECH\processed_images"
img_size = (128, 128)
batch_size = 32

In [2]:
# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

In [3]:
# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)

Found 52030 images belonging to 12 classes.


In [5]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)


Found 52030 images belonging to 12 classes.


In [6]:
 #Load the VGG16 model without the top layers
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 42s 1us/step


In [7]:
# Freeze the base model's layers to retain pre-trained features
for layer in base_model.layers:
    layer.trainable = False

In [8]:

# Add custom layers for classification
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(12, activation="softmax")  # Adjust the output classes as per your dataset
])

In [9]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
)


C:\Users\ggows\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1626/1626 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2293 - loss: 2.1604   

C:\Users\ggows\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [ ]:
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze the last 4 layers
    layer.trainable = True


In [ ]:
# Re-compile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
)

In [ ]:
# Evaluate the model
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

In [ ]:
# Save the fine-tuned model
model.save("fine_tuned_model.h5")
print("Model saved as 'fine_tuned_model.h5'")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
# Path to the saved model file
model_path = "fine_tuned_model.h5"  # Replace with your model file path

# Load the trained model
model = load_model(model_path)
print("Model loaded successfully!")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

def predict_disease(img_path, model, img_size=(128, 128)):
    """
    Predict the disease class of the given image.

    Parameters:
    img_path (str): Path to the input image.
    model (keras.Model): Trained model for prediction.
    img_size (tuple): Target size for resizing the image.

    Returns:
    None
    """
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict the class probabilities
    prediction = model.predict(img_array)
    
    # Get the predicted class index and confidence score
    predicted_class_idx = np.argmax(prediction, axis=-1)[0]
    confidence_score = np.max(prediction)
    
    # Class labels
    class_labels = [
        'bacterial_leaf_blight',
        'bacterial_leaf_streak',
        'bacterial_panicle_blight',
        'blast',
        'brown_spot',
        'dead_heart',
        'downy_mildew',
        'hispa',
        'normal',
        'tungro'
    ]
    
    # Output the result
    predicted_class = class_labels[predicted_class_idx]
    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence Score: {confidence_score:.2f}")
    
    # Display the input image with prediction
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence_score:.2f}")
    plt.axis('off')
    plt.show()


In [ ]:
img_path = r"C:\AGROTECH\processed_images\disease\100023.jpg"# Replace with the actual test image path
predict_disease(img_path)

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history_fine_tune.history['accuracy'], label='Train Accuracy')
plt.plot(history_fine_tune.history['val_accuracy'], label='Validation Accuracy')

plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history_fine_tune.history['loss'], label='Train Loss')
plt.plot(history_fine_tune.history['val_loss'], label='Validation Loss')

plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()